In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
cd drive/My Drive/DSGA1011_PROJECT/OpenNMT-py

/content/drive/My Drive/DSGA1011_PROJECT/OpenNMT-py


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
path_to_data = 'data/'

en_file = 'europarl-v7.es-en.en'
es_file = 'europarl-v7.es-en.es'

In [ ]:
def load_data(filename):
    file = open(filename, encoding='utf-8')
    text = file.read()
    file.close()
    text = text.strip().split('\n')
    return text

In [ ]:
en_europarl = load_data(path_to_data + en_file)
es_europarl = load_data(path_to_data + es_file)

In [ ]:
assert(len(en_europarl) == len(es_europarl))

In [ ]:
# Resource : https://machinelearningmastery.com/prepare-french-english-dataset-machine-translation/
def clean_data(en_europarl, es_europarl):
    remove = []
    for i in range(len(en_europarl)):
        en_sent = en_europarl[i]
        es_sent = es_europarl[i]
        if not any(word.isalpha() for word in en_sent):
            remove.append(i)
        elif not any(word.isalpha() for word in es_sent):
            remove.append(i)
    
    for idx in reversed(remove): 
        en_europarl.pop(idx)
        es_europarl.pop(idx)

    return en_europarl, es_europarl

In [ ]:
en_europarl_cleaned, es_europarl_cleaned = clean_data(en_europarl, es_europarl)

In [ ]:
assert(len(en_europarl_cleaned) == len(es_europarl_cleaned))

In [ ]:
en_europarl_cleaned_sample = en_europarl_cleaned[:100000]
es_europarl_cleaned_sample = es_europarl_cleaned[:100000]

In [ ]:
# For future 
X_train, X_test, y_train, y_test = train_test_split(en_europarl_cleaned_sample, es_europarl_cleaned_sample, test_size=0.2, random_state=1)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=1)

In [ ]:
with open(path_to_data + 'europarl_es_train.txt', 'w') as f: 
    for line in y_train: 
        f.write(line + '\n')

In [ ]:
!pip install -r requirements.opt.txt

In [ ]:
!pip install configargparse
#!pip install torch==1.7.0
!pip install git+https://github.com/pytorch/text

     |████████████████████████████████| 51kB 7.3MB/s 
  Created wheel for configargparse: filename=ConfigArgParse-1.2.3-cp36-none-any.whl size=19329 sha256=b336d58aed7cbb4aa8d9048bf0bcacdf46367f08fa66fa48b452a3d200e41cdb
  Stored in directory: /root/.cache/pip/wheels/bd/d6/53/034032da9498bda2385cd50a51a289e88090b5da2d592b1fdf
Successfully built configargparse
  Cloning https://github.com/pytorch/text to /tmp/pip-req-build-v51sty2s
  Running command git clone -q https://github.com/pytorch/text /tmp/pip-req-build-v51sty2s
  Running command git submodule update --init --recursive -q
  Created wheel for torchtext: filename=torchtext-0.9.0a0+d6d7f20-cp36-cp36m-linux_x86_64.whl size=7049410 sha256=2fec40c047c6a35deb4588eb584763dbf6b6f6979702423ec1e3ce753ed8ec9a
  Stored in directory: /tmp/pip-ephem-wheel-cache-hp_a48wy/wheels/73/14/71/ed033fd999ae4933e17df3e91be2014e61c2f312a88a164ff5
Successfully built torchtext
  Found existing installation: torchtext 0.3.1
    Uninstalling torchtext-0.3.1

In [ ]:
!python preprocess.py -overwrite -train_src data/europarl_es_train.txt -train_tgt data/europarl_en_train.txt -valid_src data/europarl_es_val.txt -valid_tgt data/europarl_en_val.txt -save_data data/demo

[2020-11-26 00:01:13,904 INFO] Extracting features...
[2020-11-26 00:01:13,906 INFO]  * number of source features: 0.
[2020-11-26 00:01:13,906 INFO]  * number of target features: 0.
[2020-11-26 00:01:13,906 INFO] Building `Fields` object...
/usr/local/lib/python3.6/dist-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)
/usr/local/lib/python3.6/dist-packages/torchtext/data/field.py:36: UserWarning: TextMultiField class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further informa

In [ ]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [ ]:
!python train.py --gpu_ranks 0 -data data/demo -save_model demo-model -train_steps 50000 -seed 42

[2020-12-01 06:46:23,952 INFO]  * src vocab size = 50002
[2020-12-01 06:46:23,952 INFO]  * tgt vocab size = 50004
[2020-12-01 06:46:23,952 INFO] Building model...
[2020-12-01 06:46:35,572 INFO] NMTModel(
  (encoder): RNNEncoder(
    (embeddings): Embeddings(
      (make_embedding): Sequential(
        (emb_luts): Elementwise(
          (0): Embedding(50002, 500, padding_idx=1)
        )
      )
    )
    (rnn): LSTM(500, 500, num_layers=2, dropout=0.3)
  )
  (decoder): InputFeedRNNDecoder(
    (embeddings): Embeddings(
      (make_embedding): Sequential(
        (emb_luts): Elementwise(
          (0): Embedding(50004, 500, padding_idx=1)
        )
      )
    )
    (dropout): Dropout(p=0.3, inplace=False)
    (rnn): StackedLSTM(
      (dropout): Dropout(p=0.3, inplace=False)
      (layers): ModuleList(
        (0): LSTMCell(1000, 500)
        (1): LSTMCell(500, 500)
      )
    )
    (attn): GlobalAttention(
      (linear_in): Linear(in_features=500, out_features=500, bias=False)
     

In [ ]:
!python translate.py -model demo-model_step_100000.pt -src data/europarl_es_test.txt -tgt data/europarl_en_test.txt -output baseline_result -gpu 0 -replace_unk

In [ ]:
wget https://raw.githubusercontent.com/moses-smt/mosesdecoder/master/scripts/generic/multi-bleu.perl
perl multi-bleu.perl challenger/valid.zh \
     < challenger/valid_pred.58.79